<a href="https://colab.research.google.com/github/elhamod/BA820/blob/main/Clustering/Bank_Transaction_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Course: BA820 - Unsupervised and Unstructured ML**

**Notebook created by: Mohannad Elhamod**

In [219]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [220]:
import pandas as pd
import datetime

#Analyzing Bank Transactions

As a data analyst at a bank, you are given the following dataset to better understand the bank's client behavior and extract insights and interesting observations.

In [221]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "bank_transactions.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "shivamb/bank-customer-segmentation",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

# Take a random subset. (How do you know X% is enough?)
df = df.sample(frac=0.1, random_state=42)

display(df)

Using Colab cache for faster access to the 'bank-customer-segmentation' dataset.


,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
892845,T892846,C5725279,8/9/84,M,JASPUR,84447.82,7/9/16,183210,11858.0
444994,T444995,C4588538,2/1/94,M,GURGAON,12549.49,16/8/16,161300,250.0
614896,T614897,C2416476,14/11/90,M,NEW DELHI,33607.65,26/8/16,102007,3360.0
457036,T457037,C5940151,15/9/90,M,HYDERABAD,38238.86,21/8/16,110438,250.0
997441,T997442,C5922989,27/11/88,M,PURBO MEDINIPUR,9506.85,14/9/16,90810,33.0
...,...,...,...,...,...,...,...,...,...
893532,T893533,C5329328,3/8/81,F,NEW DELHI,24720.52,7/9/16,161126,1200.0
667265,T667266,C4640853,15/4/89,M,DELHI,3033.73,31/8/16,81403,100.0
450371,T450372,C5035929,25/6/87,F,UDUPI,511.91,16/8/16,174740,627.0
427874,T427875,C3924762,10/9/79,F,KALWA KALWA THANE,28648.28,17/8/16,212547,1500.0


In [222]:
df.describe()

,CustAccountBalance,TransactionTime,TransactionAmount (INR)
count,1.046290e+05,104857.000000,104857.000000
mean,1.154616e+05,157072.778279,1600.546295
std,8.709730e+05,51399.599862,6285.266776
min,0.000000e+00,1.000000,0.000000
25%,4.740200e+03,124049.000000,165.000000
50%,1.682998e+04,164302.000000,463.000000
75%,5.745586e+04,200043.000000,1200.000000
max,1.150355e+08,235958.000000,600008.320000


##Preprocessing

If we are to perform behavioral segmentation, preprocessing ay include questions such as:
- Which features are useful to the task?
- Any transformations needed?
- Are all values valid?

In [223]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104857 entries, 892845 to 1042883
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   TransactionID            104857 non-null  object 
 1   CustomerID               104857 non-null  object 
 2   CustomerDOB              104513 non-null  object 
 3   CustGender               104751 non-null  object 
 4   CustLocation             104847 non-null  object 
 5   CustAccountBalance       104629 non-null  float64
 6   TransactionDate          104857 non-null  object 
 7   TransactionTime          104857 non-null  int64  
 8   TransactionAmount (INR)  104857 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 8.0+ MB


In [224]:
# Remove null values
df = df.dropna()

Check whether date values are valid

In [225]:
# Convert all DOB to format MMDDYYYY
df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'])
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

# df['CustomerDOB'] = df['CustomerDOB'].dt.strftime('%m%d%Y')
# df['TransactionDate'] = df['TransactionDate'].dt.strftime('%m%d%Y')

# Show me the range of DOB dates
df['CustomerDOB'].min(), df['CustomerDOB'].max()

/tmp/ipython-input-1995574973.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'])
/tmp/ipython-input-1995574973.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])


(Timestamp('1800-01-01 00:00:00'), Timestamp('2075-12-31 00:00:00'))

In [226]:
# Remove TransactionID as it is redundant
df = df.drop('TransactionID', axis=1)

# Remove the C from CustomerID
df['CustomerID'] = df['CustomerID'].str.replace('C', '')


# birth dates between 1900s and 2017
df = df[
    (df['CustomerDOB'] >= '1900-01-01') &
    (df['CustomerDOB'] <= '2017-01-01')
]



display(df)

,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
892845,5725279,1984-08-09,M,JASPUR,84447.82,2016-07-09,183210,11858.0
444994,4588538,1994-02-01,M,GURGAON,12549.49,2016-08-16,161300,250.0
614896,2416476,1990-11-14,M,NEW DELHI,33607.65,2016-08-26,102007,3360.0
457036,5940151,1990-09-15,M,HYDERABAD,38238.86,2016-08-21,110438,250.0
997441,5922989,1988-11-27,M,PURBO MEDINIPUR,9506.85,2016-09-14,90810,33.0
...,...,...,...,...,...,...,...,...
893532,5329328,1981-03-08,F,NEW DELHI,24720.52,2016-07-09,161126,1200.0
667265,4640853,1989-04-15,M,DELHI,3033.73,2016-08-31,81403,100.0
450371,5035929,1987-06-25,F,UDUPI,511.91,2016-08-16,174740,627.0
427874,3924762,1979-10-09,F,KALWA KALWA THANE,28648.28,2016-08-17,212547,1500.0


Let's get the interesting subset of numerical feature and prepare it for clustering

In [227]:
# Only keep numeric values (it will drop the TransactionID)
df_numeric = df.select_dtypes(include=['number'])
df_numeric.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86608 entries, 892845 to 1042883
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   CustAccountBalance       86608 non-null  float64
 1   TransactionTime          86608 non-null  int64  
 2   TransactionAmount (INR)  86608 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 2.6 MB


In [228]:
display(df_numeric)

,CustAccountBalance,TransactionTime,TransactionAmount (INR)
892845,84447.82,183210,11858.0
444994,12549.49,161300,250.0
614896,33607.65,102007,3360.0
457036,38238.86,110438,250.0
997441,9506.85,90810,33.0
...,...,...,...
893532,24720.52,161126,1200.0
667265,3033.73,81403,100.0
450371,511.91,174740,627.0
427874,28648.28,212547,1500.0


Let's process some features to either them numerically feasable for clustering

In [229]:
# convert df customerDoB to numbers
df_numeric['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'])

# transaction dates
df_numeric['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

# time as a datetime object
df_numeric['TransactionTime'] = pd.to_datetime(
    df_numeric['TransactionTime'].astype(str).str.zfill(6),
    format="%H%M%S"
).dt.time

# convert CustomerDOB to age in years
df_numeric['CustomerAge'] = ((df_numeric['TransactionDate'] - df_numeric['CustomerDOB']).dt.days / 365.25).astype(int)
df_numeric.drop('CustomerDOB', axis=1, inplace=True)


display(df_numeric)

,CustAccountBalance,TransactionTime,TransactionAmount (INR),TransactionDate,CustomerAge
892845,84447.82,18:32:10,11858.0,2016-07-09,31
444994,12549.49,16:13:00,250.0,2016-08-16,22
614896,33607.65,10:20:07,3360.0,2016-08-26,25
457036,38238.86,11:04:38,250.0,2016-08-21,25
997441,9506.85,09:08:10,33.0,2016-09-14,27
...,...,...,...,...,...
893532,24720.52,16:11:26,1200.0,2016-07-09,35
667265,3033.73,08:14:03,100.0,2016-08-31,27
450371,511.91,17:47:40,627.0,2016-08-16,29
427874,28648.28,21:25:47,1500.0,2016-08-17,36


In [230]:
df_numeric.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86608 entries, 892845 to 1042883
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   CustAccountBalance       86608 non-null  float64       
 1   TransactionTime          86608 non-null  object        
 2   TransactionAmount (INR)  86608 non-null  float64       
 3   TransactionDate          86608 non-null  datetime64[ns]
 4   CustomerAge              86608 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 4.0+ MB


In [231]:
df_numeric.describe()

,CustAccountBalance,TransactionAmount (INR),TransactionDate,CustomerAge
count,8.660800e+04,86608.000000,86608,86608.000000
mean,7.513220e+04,1272.554387,2016-07-25 10:18:26.669129984,28.112415
min,0.000000e+00,0.000000,2016-01-08 00:00:00,0.000000
25%,4.158352e+03,144.000000,2016-06-09 00:00:00,24.000000
50%,1.430624e+04,393.000000,2016-08-20 00:00:00,27.000000
75%,4.513553e+04,1000.000000,2016-09-09 00:00:00,32.000000
max,2.797959e+07,600008.320000,2016-12-09 00:00:00,40.000000
std,3.399854e+05,4980.650537,NaN,5.003353


##Normalize

In [233]:
# Normalizing the numeric features
from sklearn.preprocessing import StandardScaler

df_numeric_normalized = df_numeric.copy()

# Date becomes an integer
df_numeric_normalized['TransactionDate'] = df_numeric_normalized['TransactionDate'].astype('int64')
# convert TransactionTime to seconds
df_numeric_normalized['TransactionTime'] = pd.to_timedelta(df_numeric['TransactionTime'].astype(str)).dt.total_seconds()

#Normalize
scaler = StandardScaler()
df_numeric_normalized = pd.DataFrame(scaler.fit_transform(df_numeric_normalized), columns=df_numeric_normalized.columns)

# df_numeric_normalized should have same indices as df_numeric
df_numeric_normalized.index = df_numeric.index

display(df_numeric_normalized)

,CustAccountBalance,TransactionTime,TransactionAmount (INR),TransactionDate,CustomerAge
892845,0.027400,0.496805,2.125326,-0.198912,0.577133
444994,-0.184076,0.051187,-0.205307,0.261155,-1.221671
614896,-0.122137,-1.078761,0.419113,0.382226,-0.622069
457036,-0.108515,-0.936217,-0.205307,0.321691,-0.622069
997441,-0.193025,-1.309148,-0.248875,0.612260,-0.222335
...,...,...,...,...,...
893532,-0.148277,0.046170,-0.014567,-0.198912,1.376602
667265,-0.212065,-1.482432,-0.235423,0.442761,-0.222335
450371,-0.219482,0.354314,-0.129613,0.261155,0.177399
427874,-0.136724,1.052733,0.045666,0.273262,1.576469


In [234]:
df_numeric_normalized.describe()

,CustAccountBalance,TransactionTime,TransactionAmount (INR),TransactionDate,CustomerAge
count,8.660800e+04,8.660800e+04,8.660800e+04,8.660800e+04,8.660800e+04
mean,3.183200e-17,4.381002e-17,-4.700963e-17,-1.561146e-14,9.385517e-17
std,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00
min,-2.209878e-01,-3.064349e+00,-2.555011e-01,-2.414503e+00,-5.618748e+00
25%,-2.087568e-01,-6.231503e-01,-2.265891e-01,-5.621240e-01,-8.219365e-01
50%,-1.789086e-01,1.708099e-01,-1.765953e-01,3.095836e-01,-2.223351e-01
75%,-8.822980e-02,7.956221e-01,-5.472296e-02,5.517246e-01,7.770006e-01
max,8.207594e+01,1.546435e+00,1.202131e+02,1.653466e+00,2.375938e+00


##Questions:

###Q1. What natural *customer* segments exist in the bank's customer base? Think about what customer descriptors are useful here.

###Q2. For each customer segment you identified, describe (1) the key insight about that segment and (2) a specific action the bank could take to benefit the business and/or the customer.

###Q3. Which customers show unusual patterns? Do these customers form an opportunity or risk? What recommendations do you make?

###Q4. What demographic profiling could help predict an applicant's segment reliably? What policy would you (or would you not) implement based on such profiling?

Be careful of discrimination..

###Q5. Profile the *transaction* in this dataset. Think about what transaction descriptors are useful here.